Korzystając z tabeli “connections”, która znajduje się w bazie danych MySQL, do której zdobyliśmy dostęp w S03E03, wyznacz najkrótszą drogę od Rafała do Barbary. Wspomniana tabela zawiera listę połączeń między osobami. Drogę zwróć jako imiona kolejnych osób na drodze, począwszy od imienia Rafał, a kończąc na Barbara. Zastosuj przecinek jako separator imion. Raport zgłoś jako zadanie o nazwie connections. Wykorzystaj do tego zadania bazę grafową. Sugerujemy użycie NEO4j, ale możesz użyć tej, która będzie dla CIebie wygodna.

Co należy zrobić w zadaniu?

Pobierz zawartość tabel Users users (tutaj są imiona oraz ID) oraz Connections connections (tutaj jest mapowanie powiązań między ID). Możesz zapisać je u siebie lokalnie, ponieważ ich zawartość się nie zmienia. Twój automat nie musi odpytywać bazy o dane na bieżąco.

Tabela “connections” zawiera dwa ID użytkowników. Źródłowego i docelowego. Oznaczenie pary “17,29” oznacza więc, że użytkownik od ID=17 zna użytkownika o ID=29

Dla uproszczenia zadania zbudowaliśmy relację jednokierunkową. Czyli jeśli Marian zna Stefana, to w bazie prawdopodobnie nie znajdziesz informacji o tym, że Stefan zna Mariana. Nie musisz obsługiwać takiej sytuacji. Aby jeszcze bardziej uprościć analizę, imiona osób są unikalne.

Zbuduj strukturę w bazie grafowej, która będzie pokazywać, która osoba kogo zna. Załóż oczywiście, że jedna osoba może znać wiele innych osób. Nie są to połączenia jeden do jednego.

Odpytaj bazę o najkrótszą drogę od Rafała do Barbary

Odpowiedź wyślij w formie stringa podobnego do tego poniżej do /report jako zadanie connections.

Rafał, Andrzej, Stefan, Marian, Artur, Barbara


Wielkość liter nie ma znaczenia.


Uwaga: celem zadania jest praktyczne zaznajomienie Cię z obsługą baz grafowych. Bezpośrednie zastosowanie LLMa w tym zadaniu nie jest konieczne. Bazy tego typu przydadzą Ci się do lepszej organizacji zbioru informacji w przyszłości.

In [1]:
import os
import requests
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()

personal_api_key = os.getenv("PERSONAL_API_KEY")
print(personal_api_key)
URI = os.getenv("NEO4J_URI")
print(URI)
AUTH = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))
print(AUTH)

1400cbf0-b7dd-49ab-9342-6ad8fd26ba69
neo4j://localhost
('neo4j', 'zaq1@#$%')


In [3]:
querry_json_user = {
    "task": "database",
    "apikey": personal_api_key,
    "query": "select * from users limit 1"
}
get_data_users = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_user)
print(get_data_users.json())

{'reply': [{'id': '1', 'username': 'Adrian', 'access_level': 'user', 'is_active': '1', 'lastlog': '2023-06-12'}], 'error': 'OK'}


In [4]:
querry_json_tables = {
    "task": "database",
    "apikey": personal_api_key,
    "query": "show tables"
}
get_data_tables = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_tables)
print(get_data_tables.json())

{'reply': [{'Tables_in_banan': 'connections'}, {'Tables_in_banan': 'correct_order'}, {'Tables_in_banan': 'datacenters'}, {'Tables_in_banan': 'users'}], 'error': 'OK'}


In [5]:
get_data_tables_struct = {}
for table in get_data_tables.json()["reply"]:    
    print(table["Tables_in_banan"])
    querry_json_tables_struct = {
        "task": "database",
        "apikey": personal_api_key,
        "query": f"show create table {table["Tables_in_banan"]}"
    }
    get_data_tables_struct[table["Tables_in_banan"]] = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_tables_struct).json()
print(get_data_tables_struct)


connections


correct_order
datacenters
users
{'connections': {'reply': [{'Table': 'connections', 'Create Table': 'CREATE TABLE `connections` (\n  `user1_id` int(11) NOT NULL,\n  `user2_id` int(11) NOT NULL,\n  PRIMARY KEY (`user1_id`,`user2_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci'}], 'error': 'OK'}, 'correct_order': {'reply': [{'Table': 'correct_order', 'Create Table': 'CREATE TABLE `correct_order` (\n  `base_id` int(11) DEFAULT NULL,\n  `letter` char(1) DEFAULT NULL,\n  `weight` int(11) DEFAULT 0\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci'}], 'error': 'OK'}, 'datacenters': {'reply': [{'Table': 'datacenters', 'Create Table': 'CREATE TABLE `datacenters` (\n  `dc_id` int(11) DEFAULT NULL,\n  `location` varchar(30) NOT NULL,\n  `manager` int(11) NOT NULL DEFAULT 31,\n  `is_active` int(11) DEFAULT 0\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci'}], 'error': 'OK'}, 'users': {'reply': [{'Table': 'users', 'Create Table': "CREA

In [13]:
querry_json_users_data = {
    "task": "database",
    "apikey": personal_api_key,
    "query": "select * from users"
}
get_data_users_data = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_users_data)
print(json.dumps(get_data_users_data.json(), indent=4, ensure_ascii=False))

{
    "reply": [
        {
            "id": "1",
            "username": "Adrian",
            "access_level": "user",
            "is_active": "1",
            "lastlog": "2023-06-12"
        },
        {
            "id": "2",
            "username": "Monika",
            "access_level": "user",
            "is_active": "1",
            "lastlog": "2023-09-29"
        },
        {
            "id": "3",
            "username": "Azazel",
            "access_level": "removed",
            "is_active": "0",
            "lastlog": "2026-11-05"
        },
        {
            "id": "4",
            "username": "Robert",
            "access_level": "user",
            "is_active": "1",
            "lastlog": "2024-11-06"
        },
        {
            "id": "5",
            "username": "Aleksandra",
            "access_level": "user",
            "is_active": "1",
            "lastlog": "2023-03-27"
        },
        {
            "id": "6",
            "username": "Michał",
         

In [14]:
querry_json_connections_data = {
    "task": "database",
    "apikey": personal_api_key,
    "query": "select * from connections"
}
get_data_connections_data = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_connections_data)
print(json.dumps(get_data_connections_data.json(), indent=4, ensure_ascii=False))


{
    "reply": [
        {
            "user1_id": "1",
            "user2_id": "5"
        },
        {
            "user1_id": "1",
            "user2_id": "20"
        },
        {
            "user1_id": "1",
            "user2_id": "54"
        },
        {
            "user1_id": "2",
            "user2_id": "14"
        },
        {
            "user1_id": "2",
            "user2_id": "45"
        },
        {
            "user1_id": "3",
            "user2_id": "6"
        },
        {
            "user1_id": "3",
            "user2_id": "31"
        },
        {
            "user1_id": "3",
            "user2_id": "77"
        },
        {
            "user1_id": "4",
            "user2_id": "2"
        },
        {
            "user1_id": "4",
            "user2_id": "34"
        },
        {
            "user1_id": "4",
            "user2_id": "83"
        },
        {
            "user1_id": "6",
            "user2_id": "95"
        },
        {
            "user1_id": "7",

In [15]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [42]:
# Convert the JSON data to lists of dictionaries for easier processing
users_data = get_data_users_data.json()["reply"]
connections_data = get_data_connections_data.json()["reply"]

# Prepare Cypher queries
create_users = []
create_connections = []
# print(json.dumps(users_data, indent=4, ensure_ascii=False))
# print(json.dumps(connections_data, indent=4, ensure_ascii=False))
# Prepare user nodes
for user in users_data:
    create_users.append(
        f"CREATE (u{user['id']}:User {{id: {user['id']}, username: '{user['username']}'}})"
    )

# Prepare connections
for conn in connections_data:
    create_connections.append(
        f"MATCH (u1:User {{id: {conn['user1_id']}}}),"
        f"(u2:User {{id: {conn['user2_id']}}})"
        f"CREATE (u1)-[:KNOWS]->(u2)"
        f"CREATE (u2)-[:KNOWS]->(u1)"
    )

# Execute queries with Neo4j
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        # First clear the database
        session.run("MATCH (n) DETACH DELETE n")
        
        # Create user nodes
        for query in create_users:
            session.run(query)
            
        # Create connections
        for query in create_connections:
            session.run(query)
            
        # Find shortest path between Rafał and Barbara
        result = session.run("""
            MATCH (start:User {username : 'Rafał'}),
                (end:User {username : 'Barbara'}),
                p = shortestPath((start)-[:KNOWS*]-(end))
            RETURN [node IN nodes(p) | node.username] as path
        """)
        
        path = result.single()
        if path:
            print("Shortest path found:", path)
        else:
            print("No path found between Rafał and Barbara")


Shortest path found: <Record path=['Rafał', 'Azazel', 'Aleksander', 'Barbara']>


In [51]:
answer = ""
for node in path["path"]:
    answer += node + ", "
answer = answer[:-2]
print(answer)

answer_json = {
    "task": "connections",
    "apikey": personal_api_key,
    "answer": answer
}
answer_url = "https://centrala.ag3nts.org/report"
answer_response = requests.post(answer_url, json=answer_json)
print(answer_response.request.body)  # Displays the request body

# Assuming `login_response` is the response object from the login request
print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
print("Response Body:", answer_response.text.encode('utf-8'))       # Displays the response content as a string    

Rafał, Azazel, Aleksander, Barbara
b'{"task": "connections", "apikey": "1400cbf0-b7dd-49ab-9342-6ad8fd26ba69", "answer": "Rafa\\u0142, Azazel, Aleksander, Barbara"}'
Status Code: 200
Response Body: b'{\n    "code": 0,\n    "message": "{{FLG:SPYWEB}}"\n}'
